In [0]:
dbutils.widgets.get("catalog")
catalog = dbutils.widgets.get("catalog")

dbutils.widgets.get("gold_schema")
gold_schema = dbutils.widgets.get("gold_schema")

In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.sm_fact_coffee_sales_fallback
(
  product_name COMMENT 'Product Name',
  product_category COMMENT 'Product Category',
  product_subcategory COMMENT 'Product Subcategory',
  day_of_week COMMENT 'Day of Week',
  date COMMENT 'Date',
  store_name COMMENT 'Store Name',
  store_online COMMENT 'Store Online',
  store_latitude COMMENT 'Store Latitude',
  store_longitude COMMENT 'Store Longitude',
  total_net_revenue_usd COMMENT 'Total net revenue in USD',
  total_cost_of_goods COMMENT 'Total cost of goods sold in USD',
  total_net_profit COMMENT 'Total net profit in USD'
)
WITH METRICS
LANGUAGE YAML
AS $$
version: 1.1

source: sunny_bay_roastery.gold.fact_coffee_sales

joins:
  - name: product
    source: sunny_bay_roastery.gold.dim_product
    on: source.product_key = product.product_key
  - name: date
    source: sunny_bay_roastery.gold.dim_date
    on: source.date_key = date.date_key
  - name: store
    source: sunny_bay_roastery.gold.dim_store
    on: source.store_key = store.store_key

dimensions:
  - name: product_name
    expr: product.product_name
    display_name: Product Name
  - name: product_category
    expr: product.product_category
    display_name: Product Category
  - name: product_subcategory
    expr: product.product_subcategory
    display_name: Product Subcategory
  - name: day_of_week
    expr: date.day_of_week
    display_name: Day of Week
  - name: date
    expr: date.date
    display_name: Date
  - name: store_name
    expr: store.store_name
    display_name: Store Name
  - name: store_online
    expr: store.is_online
    display_name: Store Online
  - name: store_latitude
    expr: store.latitude
    display_name: Store Latitude
  - name: store_longitude
    expr: store.longitude
    display_name: Store Longitude

measures:
  - name: total_net_revenue_usd
    expr: SUM(net_revenue_usd)
  - name: total_cost_of_goods
    expr: SUM(cost_of_goods_usd)
  - name: total_net_profit
    expr: measure(total_net_revenue_usd) - measure(total_cost_of_goods)
$$""")